In [43]:
#!/usr/bin/env python3
"""
ts_forecast_attention.py

Self-contained multi-step time-series forecasting example using:
- Synthetic multivariate data (>=5000 timesteps)
- Preprocessing (min-max scaling)
- Seq2Seq LSTM encoder-decoder with additive attention
- Training loop, evaluation (RMSE, MAE, MASE)
- Monte-Carlo dropout for prediction intervals
- Attention visualization

Dependencies:
- numpy
- torch
- matplotlib

Tested with: Python 3.9+, PyTorch 1.10+ (CPU or GPU)
"""
import math
import random
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from typing import Tuple

# -----------------------------
# Utilities: reproducibility
# -----------------------------
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)

# -----------------------------
# 1) Generate synthetic dataset
# -----------------------------
def generate_multivariate_series(n_steps=6000, n_series=3, noise_std=0.2):
    """
    Create synthetic multivariate time series mixing sinusoids, trends, and autoregression.
    Returns array shape (n_steps, n_series)
    """
    t = np.arange(n_steps)
    data = []
    for s in range(n_series):
        freq = 0.01 + 0.01 * s
        phase = s * 0.5
        trend = 0.0005 * t * (1 + 0.2 * s)
        seasonal = np.sin(2 * np.pi * freq * t + phase) + 0.5 * np.sin(2 * np.pi * freq * 3 * t + phase/2)
        noise = np.random.normal(scale=noise_std, size=n_steps)
        ar = np.zeros(n_steps)
        for i in range(2, n_steps):
            ar[i] = 0.6 * ar[i-1] - 0.2 * ar[i-2] + 0.05 * seasonal[i-1]
        series = seasonal + trend + ar + noise
        data.append(series)
    data = np.stack(data, axis=1)  # shape (n_steps, n_series)
    return data

# -----------------------------
# 2) Preprocessing and windows
# -----------------------------
class MinMaxScaler:
    def fit(self, data: np.ndarray):
        self.min = data.min(axis=0)
        self.max = data.max(axis=0)
        # avoid division by zero
        self.range = np.where(self.max - self.min == 0, 1.0, self.max - self.min)
    def transform(self, data: np.ndarray):
        return (data - self.min) / self.range
    def inverse_transform(self, data: np.ndarray):
        return data * self.range + self.min

def create_windows(data: np.ndarray, in_len: int, out_len: int, step=1):
    """
    data: (T, features)
    returns X (N, in_len, features), Y (N, out_len, features)
    """
    T, F = data.shape
    Xs, Ys = [], []
    for start in range(0, T - in_len - out_len + 1, step):
        Xs.append(data[start : start + in_len])
        Ys.append(data[start + in_len : start + in_len + out_len])
    X = np.stack(Xs)  # (N, in_len, F)
    Y = np.stack(Ys)  # (N, out_len, F)
    return X, Y

# -----------------------------
# Dataset class
# -----------------------------
class SeqDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X.astype(np.float32)
        self.Y = Y.astype(np.float32)
    def __len__(self):
        return len(self.X)
    def __getitem__(self, idx):
        return self.X[idx], self.Y[idx]

# -----------------------------
# 3) Seq2Seq LSTM with Attention
# -----------------------------
class EncoderLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers=1, dropout=0.1):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers=num_layers, batch_first=True, dropout=dropout)
    def forward(self, x):
        # x: (batch, seq, features)
        out, (h, c) = self.lstm(x)
        # out: (batch, seq, hidden)
        return out, (h, c)

class AdditiveAttention(nn.Module):
    def __init__(self, enc_dim, dec_dim, attn_dim):
        super().__init__()
        self.W_enc = nn.Linear(enc_dim, attn_dim, bias=False)
        self.W_dec = nn.Linear(dec_dim, attn_dim, bias=False)
        self.v = nn.Linear(attn_dim, 1, bias=False)
    def forward(self, enc_outputs, dec_hidden):
        # enc_outputs: (batch, seq_enc, enc_dim)
        # dec_hidden: (batch, dec_dim)  (we'll use hidden state from decoder)
        # returns context (batch, enc_dim) and attn_weights (batch, seq_enc)
        # compute scores
        enc_proj = self.W_enc(enc_outputs)  # (B, S, A)
        dec_proj = self.W_dec(dec_hidden).unsqueeze(1)  # (B, 1, A)
        e = torch.tanh(enc_proj + dec_proj)  # (B, S, A)
        scores = self.v(e).squeeze(-1)  # (B, S)
        attn_weights = torch.softmax(scores, dim=1)  # (B, S)
        context = torch.bmm(attn_weights.unsqueeze(1), enc_outputs).squeeze(1)  # (B, enc_dim)
        return context, attn_weights

class DecoderLSTMWithAttn(nn.Module):
    def __init__(self, input_dim, enc_dim, dec_dim, out_dim, attn_dim, num_layers=1, dropout=0.1):
        super().__init__()
        self.input_proj = nn.Linear(input_dim + enc_dim, dec_dim)  # combine previous target and context
        self.lstm = nn.LSTM(dec_dim, dec_dim, num_layers=num_layers, batch_first=True, dropout=dropout)
        self.attn = AdditiveAttention(enc_dim, dec_dim, attn_dim)
        self.out = nn.Linear(dec_dim, out_dim)
        self.dropout = nn.Dropout(dropout)
    def forward_step(self, prev_y, enc_outputs, hidden):
        # prev_y: (B, features)  previous target / input to decoder
        # enc_outputs: (B, seq, enc_dim)
        # hidden: (h, c) where h: (num_layers, B, dec_dim)
        dec_hidden = hidden[0][-1]  # last layer hidden (B, dec_dim)
        context, attn_weights = self.attn(enc_outputs, dec_hidden)  # (B, enc_dim), (B, S)
        # combine prev_y and context
        combined = torch.cat([prev_y, context], dim=1)  # (B, features+enc_dim)
        dec_input = torch.tanh(self.input_proj(combined)).unsqueeze(1)  # (B,1,dec_dim)
        out, hidden = self.lstm(dec_input, hidden)
        out = out.squeeze(1)  # (B, dec_dim)
        out = self.dropout(out)
        y_pred = self.out(out)  # (B, out_dim)
        return y_pred, hidden, attn_weights

    def forward(self, enc_outputs, dec_init_hidden, targets=None, teacher_forcing_ratio=0.5):
        """
        enc_outputs: (B, seq_enc, enc_dim)
        dec_init_hidden: tuple(h, c) where each is (num_layers, B, dec_dim)
        targets: (B, out_len, out_dim) or None
        returns preds (B, out_len, out_dim), attn_weights over time (B, out_len, seq_enc)
        """
        B = enc_outputs.size(0)
        device = enc_outputs.device
        if targets is not None:
            out_len = targets.size(1)
            out_dim = targets.size(2)
        else:
            out_len = 10
            out_dim = enc_outputs.size(2)  # fallback
        preds = []
        attns = []
        # start token: use last input step from encoder as initial prev_y (or zeros)
        # We'll let caller provide useful start; here use zeros
        prev_y = torch.zeros(B, out_dim, device=device)
        hidden = dec_init_hidden
        for t in range(out_len):
            y_pred, hidden, attn_w = self.forward_step(prev_y, enc_outputs, hidden)
            preds.append(y_pred.unsqueeze(1))
            attns.append(attn_w.unsqueeze(1))
            # decide next prev_y
            if targets is not None and random.random() < teacher_forcing_ratio:
                prev_y = targets[:, t, :].to(device)
            else:
                prev_y = y_pred.detach()
        preds = torch.cat(preds, dim=1)
        attns = torch.cat(attns, dim=1)  # (B, out_len, seq_enc)
        return preds, attns

# Full model wrapper
class Seq2SeqAttnModel(nn.Module):
    def __init__(self, input_dim, enc_dim=64, dec_dim=64, attn_dim=32, out_dim=None, dropout=0.1):
        super().__init__()
        self.encoder = EncoderLSTM(input_dim, enc_dim, num_layers=1, dropout=dropout)
        self.decoder = DecoderLSTMWithAttn(out_dim if out_dim else input_dim, enc_dim, dec_dim, out_dim if out_dim else input_dim, attn_dim, num_layers=1, dropout=dropout)
    def forward(self, src, tgt=None, teacher_forcing_ratio=0.5):
        # src: (B, seq_in, features)
        enc_out, (h, c) = self.encoder(src)
        # initialize decoder hidden from encoder final states (project sizes if needed)
        # if enc_dim != dec_dim, simple projection:
        dec_h = torch.tanh(nn.Linear(enc_out.size(2), self.decoder.lstm.hidden_size).to(src.device)(h))
        dec_c = torch.tanh(nn.Linear(enc_out.size(2), self.decoder.lstm.hidden_size).to(src.device)(c))
        # reshape dec_h/dec_c to (num_layers, B, dec_dim)
        # h/c currently (num_layers, B, enc_dim) -> after linear same shape but dec_dim
        dec_hidden = (dec_h, dec_c)
        preds, attns = self.decoder(enc_out, dec_hidden, targets=tgt, teacher_forcing_ratio=teacher_forcing_ratio)
        return preds, attns

# -----------------------------
# 4) Training and evaluation
# -----------------------------
def train_epoch(model, dataloader, optimizer, criterion, epoch, device, clip=1.0):
    model.train()
    total_loss = 0.0
    for Xb, Yb in dataloader:
        Xb = Xb.to(device)
        Yb = Yb.to(device)
        optimizer.zero_grad()
        preds, _ = model(Xb, tgt=Yb, teacher_forcing_ratio=0.6)
        loss = criterion(preds, Yb)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        optimizer.step()
        total_loss += loss.item() * Xb.size(0)
    return total_loss / len(dataloader.dataset)

def evaluate(model, dataloader, criterion, device):
    model.eval()
    total_loss = 0.0
    all_preds = []
    all_trues = []
    all_attns = []
    with torch.no_grad():
        for Xb, Yb in dataloader:
            Xb = Xb.to(device)
            Yb = Yb.to(device)
            preds, attns = model(Xb, tgt=None, teacher_forcing_ratio=0.0)  # autoregressive inference
            loss = criterion(preds, Yb)
            total_loss += loss.item() * Xb.size(0)
            all_preds.append(preds.cpu().numpy())
            all_trues.append(Yb.cpu().numpy())
            all_attns.append(attns.cpu().numpy())
    all_preds = np.concatenate(all_preds, axis=0)
    all_trues = np.concatenate(all_trues, axis=0)
    all_attns = np.concatenate(all_attns, axis=0)
    return total_loss / len(dataloader.dataset), all_preds, all_trues, all_attns

# Metrics
def rmse(y_pred, y_true):
    return np.sqrt(np.mean((y_pred - y_true) ** 2))
def mae(y_pred, y_true):
    return np.mean(np.abs(y_pred - y_true))
def mase(y_pred, y_true, train_series):
    # naive forecast: one-step naive (difference between consecutive points) averaged on training data
    # implement seasonal naive with seasonality=1 (i.e., naive persistence)
    # compute MAE of naive on training series
    # train_series: (T, features)
    naive_errors = np.abs(train_series[1:] - train_series[:-1])
    mae_naive = np.mean(naive_errors)
    mae_model = np.mean(np.abs(y_pred - y_true))
    return mae_model / (mae_naive + 1e-8)

# MC Dropout prediction intervals
def mc_dropout_predict(model, X, n_samples=50):
    """
    Keep dropout active and run forward n_samples times to get predictive distribution.
    X: tensor (B, seq_in, features)
    returns preds_samples: (n_samples, B, out_len, features)
    """
    model.train()  # activate dropout
    preds = []
    with torch.no_grad():
        for _ in range(n_samples):
            p, _ = model(X.to(device), tgt=None, teacher_forcing_ratio=0.0)
            preds.append(p.cpu().numpy())
    preds = np.stack(preds, axis=0)
    model.eval()
    return preds

# -----------------------------
# 5) Main runner
# -----------------------------
def main():
    # hyperparams
    n_steps = 6000
    n_series = 3
    in_len = 30
    out_len = 10
    batch_size = 64
    epochs = 10  # small for demo; raise to 30-80 for better performance
    lr = 1e-3
    enc_dim = 64
    dec_dim = 64
    attn_dim = 32
    dropout = 0.2

    data = generate_multivariate_series(n_steps=n_steps, n_series=n_series, noise_std=0.25)
    print("Data shape:", data.shape)  # (T, features)

    # split train/val/test (70/15/15)
    T = data.shape[0]
    train_end = int(T * 0.7)
    val_end = int(T * 0.85)
    train_data = data[:train_end]
    val_data = data[train_end:val_end]
    test_data = data[val_end:]

    scaler = MinMaxScaler()
    scaler.fit(train_data)
    data_scaled = scaler.transform(data)

    # create windows over the full scaled data, but we'll split indices to keep time order
    X_all, Y_all = create_windows(data_scaled, in_len=in_len, out_len=out_len, step=1)
    # compute split indices corresponding to time splits - find first window index >= train_end - in_len - out_len + 1 mapping
    # Simpler: map window start positions to time index
    num_windows = X_all.shape[0]
    # window start times
    starts = np.arange(0, n_steps - in_len - out_len + 1)
    train_mask = starts + in_len + out_len <= train_end
    val_mask = (starts + in_len + out_len > train_end) & (starts + in_len + out_len <= val_end)
    test_mask = starts + in_len + out_len > val_end

    X_train, Y_train = X_all[train_mask], Y_all[train_mask]
    X_val, Y_val = X_all[val_mask], Y_all[val_mask]
    X_test, Y_test = X_all[test_mask], Y_all[test_mask]
    print("Windows: train", X_train.shape, "val", X_val.shape, "test", X_test.shape)

    train_ds = SeqDataset(X_train, Y_train)
    val_ds = SeqDataset(X_val, Y_val)
    test_ds = SeqDataset(X_test, Y_test)

    train_loader = DataLoader(train_ds, batch_size=batch_size, shuffle=True, drop_last=False)
    val_loader = DataLoader(val_ds, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_ds, batch_size=batch_size, shuffle=False)

    model = Seq2SeqAttnModel(input_dim=n_series, enc_dim=enc_dim, dec_dim=dec_dim, attn_dim=attn_dim, out_dim=n_series, dropout=dropout)
    model.to(device)
    print(model)

    criterion = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    best_val = float("inf")
    for epoch in range(1, epochs + 1):
        train_loss = train_epoch(model, train_loader, optimizer, criterion, epoch, device)
        val_loss, _, _, _ = evaluate(model, val_loader, criterion, device)
        print(f"Epoch {epoch} Train Loss: {train_loss:.6f}  Val Loss: {val_loss:.6f}")
        if val_loss < best_val:
            best_val = val_loss
            torch.save(model.state_dict(), "best_seq2seq_attn.pth")

    # load best
    model.load_state_dict(torch.load("best_seq2seq_attn.pth", map_location=device))
    model.eval()

    # Evaluate on test
    test_loss, preds, trues, attns = evaluate(model, test_loader, criterion, device)
    print("Test MSE loss:", test_loss)
    # inverse scale preds and trues
    preds_inv = scaler.inverse_transform(preds.reshape(-1, n_series)).reshape(preds.shape)
    trues_inv = scaler.inverse_transform(trues.reshape(-1, n_series)).reshape(trues.shape)

    # compute aggregate metrics
    rmse_val = rmse(preds_inv, trues_inv)
    mae_val = mae(preds_inv, trues_inv)
    mase_val = mase(preds_inv, trues_inv, train_data)
    print(f"Test RMSE: {rmse_val:.6f}, MAE: {mae_val:.6f}, MASE: {mase_val:.6f}")

    # Plot sample predictions for first test batch entry
    plt.figure(figsize=(12, 6))
    sample_idx = 0
    x0 = X_test[sample_idx]  # (in_len, features) scaled
    y_true = Y_test[sample_idx]  # (out_len, features) scaled
    # model prediction (single)
    with torch.no_grad():
        x_tensor = torch.tensor(x0).unsqueeze(0).to(device)
        pred_scaled, attn = model(x_tensor, tgt=None, teacher_forcing_ratio=0.0)
        pred_scaled = pred_scaled.cpu().numpy()[0]
        attn = attn.cpu().numpy()[0]  # (out_len, seq_in)
    x_all = np.concatenate([x0, y_true], axis=0)
    x_all_inv = scaler.inverse_transform(x_all)
    pred_inv = scaler.inverse_transform(pred_scaled)
    # plot first series
    for feat in range(n_series):
        plt.subplot(n_series, 1, feat+1)
        past = scaler.inverse_transform(x0)[:, feat]
        true_future = scaler.inverse_transform(y_true)[:, feat]
        pred_future = pred_inv[:, feat]
        t_past = np.arange(-in_len, 0)
        t_fut = np.arange(0, out_len)
        plt.plot(t_past, past, label="input (past)")
        plt.plot(t_fut, true_future, 'o-', label="true future")
        plt.plot(t_fut, pred_future, 'x--', label="predicted future")
        plt.title(f"Feature {feat}")
        if feat == 0:
            plt.legend()
    plt.tight_layout()
    plt.show()

    # Plot heatmap of attention weights for sample
    plt.figure(figsize=(8, 3))
    plt.imshow(attn.T, aspect='auto', origin='lower')
    plt.xlabel("Decoder time step")
    plt.ylabel("Encoder time step")
    plt.title("Attention weights (encoder positions x decoder steps)")
    plt.colorbar()
    plt.show()

    # MC dropout intervals for that sample: run 100 MC forward passes
    x_tensor = torch.tensor(x0).unsqueeze(0)
    samples = mc_dropout_predict(model, x_tensor, n_samples=100)  # (n_samples, 1, out_len, features)
    samples = samples[:, 0]  # remove batch dim
    # compute 5th and 95th percentiles
    lower = np.percentile(samples, 5, axis=0)
    upper = np.percentile(samples, 95, axis=0)
    median = np.median(samples, axis=0)
    lower_inv = scaler.inverse_transform(lower)
    upper_inv = scaler.inverse_transform(upper)
    med_inv = scaler.inverse_transform(median)
    # plot intervals for first feature
    plt.figure(figsize=(8,4))
    feat = 0
    plt.plot(np.arange(out_len), scaler.inverse_transform(y_true)[:, feat], 'o-', label='true')
    plt.plot(np.arange(out_len), med_inv[:, feat], 'x--', label='median pred')
    plt.fill_between(np.arange(out_len), lower_inv[:, feat], upper_inv[:, feat], alpha=0.3, label='90% PI')
    plt.legend()
    plt.title("MC Dropout Prediction Intervals (sample)")
    plt.show()

    print("Done.")

if __name__ == "__main__":
    main()


Device: cpu
Data shape: (6000, 3)
Windows: train (4161, 30, 3) val (900, 30, 3) test (900, 30, 3)
Seq2SeqAttnModel(
  (encoder): EncoderLSTM(
    (lstm): LSTM(3, 64, batch_first=True, dropout=0.2)
  )
  (decoder): DecoderLSTMWithAttn(
    (input_proj): Linear(in_features=67, out_features=64, bias=True)
    (lstm): LSTM(64, 64, batch_first=True, dropout=0.2)
    (attn): AdditiveAttention(
      (W_enc): Linear(in_features=64, out_features=32, bias=False)
      (W_dec): Linear(in_features=64, out_features=32, bias=False)
      (v): Linear(in_features=32, out_features=1, bias=False)
    )
    (out): Linear(in_features=64, out_features=3, bias=True)
    (dropout): Dropout(p=0.2, inplace=False)
  )
)


C:\Users\saris\AppData\Local\Programs\Python\Python313\Lib\site-packages\torch\nn\modules\rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
  warnings.warn(


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x128 and 67x64)